<a href="https://colab.research.google.com/github/PedroEsch/Simulacao_Elementos_Finitos/blob/main/Vers%C3%A3o_final_3D_melhorada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Elemento de barra 3d (FEM3D)

In [ ]:
#@title Bibliotecas e funções { display-mode: "form" }

import numpy as np
import scipy.linalg as la
from numpy import pi, cos, sin, sqrt, arctan2
np.set_printoptions(precision=2, linewidth=100) #altera forma de array no print

# Função para calcular a matriz de rigidez local do elemento 3D:
def Klocal (E, R, L, v):

  G = E/(2*(1+v))
  A = pi*R**2
  M = p*A*L
  Ix = pi*R**4/2
  Iy = (M*R**2)/2
  Iz = (M*R**2)/2

  k_local = np.array([
    #  0        1               2               3         4             5             6        7              8               9         10            11
      [E*A/L,   0,              0,              0,        0,            0,            -E*A/L,  0,             0,              0,        0,            0             ], #0
      [0,       12*E*Iz/L**3,   0,              0,        0,            6*E*Iz/L**2,  0,       -12*E*Iz/L**3, 0,              0,        0,            6*E*Iz/L**2   ], #1
      [0,       0,              12*E*Iy/L**3,   0,        -6*E*Iy/L**2, 0,            0,       0,             -12*E*Iy/L**3,  0,        -6*E*Iy/L**2, 0             ], #2
      [0,       0,              0,              G*Ix/L,   0,            0,            0,       0,             0,              -G*Ix/L,  0,            0             ], #3
      [0,       0,              -6*E*Iy/L**2,   0,        4*E*Iy/L,     0,            0,       0,             6*E*Iy/L**2,    0,        2*E*Iy/L,     0             ], #4
      [0,       6*E*Iz/L**2,    0,              0,        0,            4*E*Iz/L,     0,       -6*E*Iz/L**2,  0,              0,        0,            2*E*Iz/L      ], #5
      [-E*A/L,  0,              0,              0,        0,            0,            E*A/L,   0,             0,              0,        0,            0             ], #6
      [0,       -12*E*Iz/L**3,  0,              0,        0,            -6*E*Iz/L**2, 0,       12*E*Iz/L**3,  0,              0,        0,            -6*E*Iz/L**2  ], #7
      [0,       0,              -12*E*Iy/L**3,  0,        6*E*Iy/L**2,  0,            0,       0,             12*E*Iy/L**3,   0,        6*E*Iy/L**2,  0             ], #8
      [0,       0,              0,              -G*Ix/L,  0,            0,            0,       0,             0,              G*Ix/L,   0,            0             ], #9
      [0,       0,              -6*E*Iy/L**2,   0,        2*E*Iy/L,     0,            0,       0,             6*E*Iy/L**2,    0,        4*E*Iy/L,     0             ], #10
      [0,       6*E*Iz/L**2,    0,              0,        0,            2*E*Iz/L,     0,       -6*E*Iz/L**2,  0,              0,        0,            4*E*Iz/L      ]  #11
  ])
  return k_local

# Função para calcular a matriz de massa local do elemento 3D:
def mlocal (R, L, p):

  A = pi*R**2
  M = p*A*L
  Ix = pi*R**4/2
  Iy = (M*R**2)/2
  Iz = (M*R**2)/2

  m_local = p*A*L*np.array([
    #  0        1                         2                         3         4                         5                         6     7                         8                         9         10                        11
    #  x1       y1                        z1                        alfa1     beta1                     theta1                    x2    y2                        z2                        alfa2     beta2                     theta2
      [1/3,     0,                        0,                        0,        0,                        0,                        1/6,  0,                        0,                        0,        0,                        0                         ], #0
      [0,       13/35 + 6*Iz/(5*A*L**2),  0,                        0,        0,                        11*L/210 + Iz/(10*A*L),   0,    9/70 - 6*Iz/(5*A*L**2),   0,                        0,        0,                        -13*L/420 + Iz/(10*A*L)   ], #1
      [0,       0,                        13/35 + 6*Iy/(5*A*L**2),  0,        -11*L/210 - Iy/(10*A*L),  0,                        0,    0,                        9/70 - 6*Iy/(5*A*L**2),   0,        13*L/420 - Iy/(10*A*L),   0                         ], #2
      [0,       0,                        0,                        Ix/(3*A), 0,                        0,                        0,    0,                        0,                        Ix/(6*A), 0,                        0                         ], #3
      [0,       0,                        -11*L/210 - Iy/(10*A*L),  0,        (L**2)/105 + 2*Iy/(15*A), 0,                        0,    0,                        -13*L/420 + Iy/(10*A*L),  0,        -L**2/140 -Iy/(30*A),     0                         ], #4
      [0,       11*L/210 + Iz/(10*A*L),   0,                        0,        0,                        (L**2)/105 + 2*Iz/(15*A), 0,    13*L/420 - Iz/(10*A*L),   0,                        0,        0,                        -L**2/140 -Iz/(30*A)      ], #5
      [1/6,     0,                        0,                        0,        0,                        0,                        1/3,  0,                        0,                        0,        0,                        0                         ], #6
      [0,       9/70 - 6*Iz/(5*A*L**2),   0,                        0,        0,                        13*L/420 - Iz/(10*A*L),   0,    13/35 + 6*Iz/(5*A*L**2),  0,                        0,        0,                        -11*L/210 - Iz/(10*A*L)   ], #7
      [0,       0,                        9/70 - 6*Iy/(5*A*L**2),   0,        -13*L/420 + Iy/(10*A*L),  0,                        0,    0,                        13/35 + 6*Iy/(5*A*L**2),  0,        11*L/210 + Iy/(10*A*L),   0                         ], #8
      [0,       0,                        0,                        Ix/(6*A), 0,                        0,                        0,    0,                        0,                        Ix/(3*A), 0,                        0                         ], #9
      [0,       0,                        13*L/420 - Iy/(10*A*L),   0,        -L**2/140 -Iy/(30*A),     0,                        0,    0,                        11*L/210 + Iy/(10*A*L),   0,        (L**2)/105 + 2*Iy/(15*A), 0                         ], #10
      [0,       -13*L/420 + Iz/(10*A*L),  0,                        0,        0,                        -L**2/140 -Iz/(30*A),     0,    -11*L/210 - Iz/(10*A*L),  0,                        0,        0,                        (L**2)/105 + 2*Iy/(15*A)  ]  #11
  ])
  return m_local

# Matriz de rotação (local para global):
def RotMat(alfa, beta, theta):

  Ra = np.array([
      [1, 0,          0         ],
      [0, cos(alfa),  sin(alfa) ],
      [0, -sin(alfa), cos(alfa) ]
  ])

  Rt = np.array([
      [cos(theta),  sin(theta), 0],
      [-sin(theta), cos(theta), 0],
      [0,           0,          1]
  ])

  Rb = np.array([
      [cos(beta),   0,  sin(beta) ],
      [0,           1,  0         ],
      [-sin(beta),  0,  cos(beta) ]
  ])

  Rquase = Ra@Rt@Rb

  R = np.zeros((12,12))

  R[:3,:3] = Rquase
  R[3:6,3:6] = Rquase
  R[6:9,6:9] = Rquase
  R[9:12,9:12] = Rquase

  return R # utilizando a matriz transposta para seguir com a equação do prof. Okabe:R @ Klocal @ R.T, nesse caso o R da equação é o R.T retornado nessa função

In [ ]:
#@title Construção do sistema global { display-mode: "form" }

# massa especifica
p = 1 # Kg/m³

# Raio da hase:
R = 5e-3 # m

# Módulo de elasticidade:
E = 210*10**9 # Pa

# Poisson
v = 0.3

nos_estaticos = np.array([
    #x      y     z
    [0,     0,    0], # nó 0
    [0,     0.3,  0], # nó 1
    [0.3,   0,    0], # nó 2
    [0.3,   0.3,  0], # nó 3
])

nos_dinamicos = np.array([
    #x      y     z
    [0,     0.15, 0], # nó 4
    [0.3,   0.15, 0], # nó 5
    [0.15,  0.15, 0]  # nó 6
    ])

nos_ref = np.append(nos_estaticos,  nos_dinamicos, axis = 0)

# Nó que recebera o esforço
no = np.array([6])
ind = no-len(nos_estaticos[0:])

ligacao_ref =  np.array([
    # nó 1  nó 2  n° de nós intermediarios
    [ 0,    4,    0], # nó ref 0 com 1
    [ 4,    1,    0], # nó ref 0 com 2
    [ 2,    5,    0], # nó ref 3 com 4
    [ 5,    3,    0], # nó ref 3 com 5
    [ 4,    6,    0], # nó ref 1 com 6
    [ 6,    5,    0]  # nó ref 6 com 4
])

nos = nos_ref
ligacao = np.zeros((1,3))

for i in range(len(ligacao_ref[0:])):
  cx = nos_ref[ligacao_ref[i,1],0] - nos_ref[ligacao_ref[i,0],0]  # comprimento em x
  cy = nos_ref[ligacao_ref[i,1],1] - nos_ref[ligacao_ref[i,0],1]  # comprimento em y
  cz = nos_ref[ligacao_ref[i,1],2] - nos_ref[ligacao_ref[i,0],2]  # comprimento em z

  ex = cx/(ligacao_ref[i,2]+1) # comprimento em x por elemento
  ey = cy/(ligacao_ref[i,2]+1) # comprimento em y por elemento
  ez = cz/(ligacao_ref[i,2]+1) # comprimento em z por elemento

  cox = nos_ref[ligacao_ref[i,0],0] # coordenada x inicial
  coy = nos_ref[ligacao_ref[i,0],1] # coordenada y inicial
  coz = nos_ref[ligacao_ref[i,0],2] # coordenada z inicial

  indice = len(nos[0:]) # quantidade de nós antes do incremento

  for j in range(ligacao_ref[i,2]):

    cox += ex
    coy += ey
    coz += ez

    nos = np.append(nos,  [[cox,coy,coz]],  axis = 0)

  for j in range(ligacao_ref[i,2]+1):
    j1 = indice - 1 + j
    j2 = indice + j
    if j == 0:
      j1 = ligacao_ref[i,0]
    if j == ligacao_ref[i,2]:
      j2 = ligacao_ref[i,1]

    ligacao = np.append(ligacao,  [[j1,j2,ligacao_ref[i,2]]], axis = 0)

ligacao = np.delete(ligacao, 0, 0)

#print(nos)
#print(ligacao)

In [ ]:
#@title Estabelecendo o sistema { display-mode: "form" }

#Matriz de rigidez do sistema (ref. global):
Ksis = np.zeros((6*len(nos[0:]), 6*len(nos[0:])))

#Matriz de massa do sistema (ref. global):
Msis = np.zeros((6*len(nos[0:]), 6*len(nos[0:])))

for i in range(len(ligacao[0:])):

  cx = nos[int(ligacao[i,1]),0] - nos[int(ligacao[i,0]),0]  # comprimento em x
  cy = nos[int(ligacao[i,1]),1] - nos[int(ligacao[i,0]),1]  # comprimento em y
  cz = nos[int(ligacao[i,1]),2] - nos[int(ligacao[i,0]),2]  # comprimento em z

  comprimento = ((cx)**2 + (cy)**2 + (cz)**2 )**(1/2) # comprimento total da ligação

  comprimento_el = comprimento/(ligacao[i,2]+1) # distancia entre nós ou comprimento do elemento

  alfa_el = 0                                             #ângulo alfa do elemento
  if ((cx**2 + cy**2)**(1/2)) == 0:
    if cz >= 0:
      beta_el = pi/2                                      #ângulo beta do elemento
    else:
      beta_el = -pi/2                                     #ângulo beta do elemento
    theta_el = 0                                          #ângulo theta do elemento
  else:
    beta_el = np.arctan((cz) / ((cx**2 + cy**2)**(1/2)))  #ângulo beta do elemento
    theta_el = np.arccos((cx) / ((cx**2 + cy**2)**(1/2))) #ângulo theta do elemento

  # Cálculo das matrizes de rigidez, massa e rotação:
  k_el= Klocal(E, R, comprimento_el, v)
  m_el = mlocal(R, comprimento_el, p)
  R_el = RotMat(alfa_el, beta_el, theta_el)

  # Transformação da matriz local para global:
  k_el_global = R_el.T @ k_el @ R_el
  m_el_global = R_el.T @ m_el @ R_el

  j1 = int(ligacao[i,0])
  j2 = int(ligacao[i,1])

  n1i = j1*6      # posição inicial do menor nó
  n1f = j1*6 + 5  # posição final do menor nó
  n2i = j2*6      # posição inicial do maior nó
  n2f = j2*6 + 5  # posição final do menor nó

  # Montagem do K1global na matriz do sistema:
  Ksis[n1i:n1f+1, n1i:n1f+1] += k_el_global[:6,    :6  ]   #kii
  Ksis[n1i:n1f+1, n2i:n2f+1] += k_el_global[:6,    6:12]   #kij
  Ksis[n2i:n2f+1, n1i:n1f+1] += k_el_global[6:12,  :6  ]   #kji
  Ksis[n2i:n2f+1, n2i:n2f+1] += k_el_global[6:12,  6:12]   #kjj
  Msis[n1i:n1f+1, n1i:n1f+1] += m_el_global[:6,    :6  ]   #mii
  Msis[n1i:n1f+1, n2i:n2f+1] += m_el_global[:6,    6:12]   #mij
  Msis[n2i:n2f+1, n1i:n1f+1] += m_el_global[6:12,  :6  ]   #mji
  Msis[n2i:n2f+1, n2i:n2f+1] += m_el_global[6:12,  6:12]   #mjj

## Resolução do sistema

In [ ]:
# Redução da matriz para determinar somente deslocamentos conhecidos:
Ksis_red = np.delete(Ksis, range(6*len(nos_estaticos[0:])), 0)
Ksis_red = np.delete(Ksis_red, range(6*len(nos_estaticos[0:])), 1)

# Vetor de forças (reduzido)
Fsis_red = np.zeros((len(Ksis_red[0])))
Fsis_red[ind[0]*6:(ind[0]+1)*6] = np.array([100,-100,-200,-40,50,60])

# Resolve sistema
Vet_u = np.linalg.solve(Ksis_red, Fsis_red)

In [ ]:
print("Deslocamentos:\nnó\t   x\t\t    y\t\t    z\t\t    rotx\t  roty\t\t  rotz")
for i in range(int(len(Vet_u)/6)):
  print(i+len(nos_estaticos[0:]),'\t',f'{Vet_u[6*i]:.2e}','\t',f'{Vet_u[6*i+1]:.2e}','\t',f'{Vet_u[6*i+2]:.2e}','\t',f'{Vet_u[6*i+3]:.2e}','\t',f'{Vet_u[6*i+4]:.2e}','\t',f'{Vet_u[6*i+5]:.2e}')

Deslocamentos:
nó	   x		    y		    z		    rotx	  roty		  rotz
4 	 2.27e-04 	 -1.47e-06 	 4.69e-04 	 -1.21e-02 	 7.51e-04 	 -8.63e-03
5 	 2.27e-04 	 1.01e-06 	 -1.38e-03 	 -1.21e-02 	 -1.11e-02 	 -4.59e-03
6 	 2.28e-04 	 -6.07e-04 	 -1.81e-03 	 -5.00e-02 	 4.21e-02 	 3.97e-02


In [ ]:
u = np.zeros((len(Ksis[0])))
u[6*len(nos_estaticos[0:]):len(Ksis[0])] = Vet_u

In [ ]:
# Cálculo das forças:
Vet_f = Ksis @ u
print("Esforços:\nnó\t    fx\t\t    fy\t\t    fz\t\t    mx\t\t    my\t\t    mz")
for i in range(int(len(Vet_f)/6)):
  print(i,'\t',f'{Vet_f[6*i]:.2e}','\t',f'{Vet_f[6*i+1]:.2e}','\t',f'{Vet_f[6*i+2]:.2e}','\t',f'{Vet_f[6*i+3]:.2e}','\t',f'{Vet_f[6*i+4]:.2e}','\t',f'{Vet_f[6*i+5]:.2e}')

Esforços:
nó	    fx		    fy		    fz		    mx		    my		    mz
0 	 4.62e+01 	 1.61e+02 	 -1.52e+02 	 -8.87e+00 	 -3.97e-01 	 -1.68e+00
1 	 -9.62e+01 	 1.61e+02 	 4.84e+01 	 -1.13e+00 	 -3.97e-01 	 -5.43e+00
2 	 1.29e+01 	 -1.11e+02 	 5.16e+01 	 6.37e+00 	 5.88e+00 	 -1.80e-02
3 	 -6.29e+01 	 -1.11e+02 	 2.52e+02 	 -1.64e+01 	 5.88e+00 	 -3.77e+00
4 	 -2.75e-13 	 0.00e+00 	 0.00e+00 	 -8.12e-16 	 -3.55e-15 	 3.55e-15
5 	 1.36e-14 	 0.00e+00 	 -1.14e-13 	 4.03e-16 	 -3.55e-15 	 0.00e+00
6 	 1.00e+02 	 -1.00e+02 	 -2.00e+02 	 -4.00e+01 	 5.00e+01 	 6.00e+01


In [ ]:
Ksis = np.delete(Ksis, range(6*len(nos_estaticos[0:])), 0)
Ksis = np.delete(Ksis, range(6*len(nos_estaticos[0:])), 1)

Msis = np.delete(Msis, range(6*len(nos_estaticos[0:])), 0)
Msis = np.delete(Msis, range(6*len(nos_estaticos[0:])), 1)

In [ ]:
w,v = la.eig(Ksis,Msis)

In [ ]:
wordenado = np.sort(w**(1/2)/(2*pi))
print(wordenado)

[  13769.95+0.j   16468.8 +0.j   20649.22+0.j   20699.91+0.j   30691.22+0.j   64003.75+0.j
   75918.1 +0.j   89586.79+0.j  100530.57+0.j  103966.19+0.j  136987.99+0.j  182536.79+0.j
  270233.42+0.j  469123.79+0.j  528095.02+0.j  714568.05+0.j  725193.31+0.j 1115844.25+0.j]


In [ ]:
print(w**(1/2)/(2*pi))

[1115844.25+0.j  469123.79+0.j   16468.8 +0.j  725193.31+0.j  714568.05+0.j  136987.99+0.j
  100530.57+0.j   20699.91+0.j   64003.75+0.j  270233.42+0.j  182536.79+0.j   75918.1 +0.j
   30691.22+0.j   20649.22+0.j   13769.95+0.j  528095.02+0.j  103966.19+0.j   89586.79+0.j]


In [ ]:
print(v)

[[ 3.50e-01  7.07e-01  5.77e-01 -4.84e-18 -1.09e-18  6.79e-21 -8.58e-20 -9.35e-18 -9.28e-21
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 3.86e-17  8.61e-17 -1.24e-16 -5.08e-02 -6.52e-02 -5.07e-05 -5.11e-05 -8.86e-05  7.45e-05
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [-0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  -1.98e-03 -3.66e-03  8.23e-03 -7.14e-02  8.81e-02 -1.00e-01 -2.37e-20 -1.69e-19  4.40e-19]
 [-0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  -1.39e-16  1.33e-15 -1.87e-14 -9.89e-14  7.82e-15  2.28e-13 -1.48e-02  7.07e-01  5.69e-01]
 [-0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  -6.11e-01 -7.07e-01  4.35e-01 -7.01e-01  8.23e-02  6.80e-01 -5.83e-18 -4.74e-17  1.24e-17]
 [-3.75e-16 -7.97e-16  1.16e-15  4.22e-01  7.04e-01  4.48e-01 -7.07e-01 -6.